<a href="https://colab.research.google.com/github/Void-Brain70/Machine_Learning/blob/main/Age_Recognition.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import numpy as np
import pandas as pd
import tensorflow
from tensorflow.keras.preprocessing.image import ImageDataGenerator 
from tensorflow.keras.models import Sequential, save_model, load_model
from tensorflow.keras.layers import Conv2D, MaxPooling2D 
from tensorflow.keras.layers import Activation, Dropout, Flatten, Dense,BatchNormalization
import numpy as np
from tensorflow.keras.callbacks import TensorBoard
import time
import matplotlib.pyplot as plt
import datetime
import cv2
import os
import tensorflow as tf
import tensorflow.keras
import cv2
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt
from sklearn.metrics import confusion_matrix
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.metrics import classification_report
print('tf version', tf.__version__)
print('keras version', tf.keras.__version__)
print('gpu is ','available' if tf.config.list_physical_devices('GPU') else 'not available')

tf version 2.5.0
keras version 2.5.0
gpu is  available


In [3]:
subdir = ['/content/drive/MyDrive/Dataset/25-30','/content/drive/MyDrive/Dataset/42-48','/content/drive/MyDrive/Dataset/60-98','/content/drive/MyDrive/Dataset/6-20']
path = '/content/drive/MyDrive/Dataset'

target_name = {'25-30': 0, '42-48': 1, '60-98': 2, '6-20': 3}
X, y = [], []
for age in subdir:
    for img_name in os.listdir(path+age):
        img = cv2.imread(path+age+'/'+img_name)
        scalled_raw_img = cv2.resize(img, (80, 80))/255.0
        X.append(scalled_raw_img)
        y.append(target_name[age])

FileNotFoundError: ignored

In [ ]:
len(X), len(y)

In [ ]:
X, y = np.array(X), np.array(y)
X.shape, y.shape

In [ ]:
y = tf.keras.utils.to_categorical(y, num_classes=4)
y.shape

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
X_train.shape, X_test.shape, y_train.shape, y_test.shape

In [ ]:
base_model = tf.keras.applications.Xception(
    include_top=False,
    weights="imagenet",
    input_shape=(80, 80, 3),
    pooling='avg',
    classes=4,
    classifier_activation="softmax",
)

In [ ]:
num_of_class = 4
epoch = 20

model = Sequential()
model.add(base_model)
model.add(Dropout(0.5))
model.add(Dense(num_of_class, activation='softmax'))


model.compile(loss ='categorical_crossentropy',
                     optimizer ='adam', 
                   metrics =['accuracy'])

In [ ]:
H = model.fit(
    x=X_train,
    y=y_train,
    batch_size=256,
    epochs=epoch,
    validation_data=(X_test, y_test)) #Model Training

In [ ]:
# Plot training & validation accuracy values
fig, ax = plt.subplots(1,1)
plt.plot(H.history["accuracy"])
plt.plot(H.history['val_accuracy'])
plt.title('Model accuracy')
plt.ylabel('Accuracy')
plt.xlabel('Epoch')
plt.legend(['Train', 'Test'], loc='upper left')
plt.show()

In [ ]:
# Plot training & validation loss values
fig, ax = plt.subplots(1,1)
plt.plot(H.history['loss'])
plt.plot(H.history['val_loss'])
plt.title('Model loss')
plt.ylabel('Loss')
plt.xlabel('Epoch')
plt.legend(['Train', 'Test'], loc='upper left')
plt.show()

In [ ]:
y_pred = np.argmax(model.predict(X_test,batch_size=1), axis=1) 

print('Classification Report')
y_classes = [np.argmax(y, axis=None, out=None) for y in y_test]
report = classification_report(y_classes, y_pred, target_names=target_name.keys())
print(report)

plt.figure(figsize=(10,10))
cm = confusion_matrix(y_classes, y_pred)
_=sns.heatmap(cm.T, annot=True, fmt='d', cbar=True, square=True, xticklabels=target_name.keys(),
             yticklabels=target_name.keys())
plt.xlabel('Truth')
plt.ylabel('Predicted')